In [ ]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.1-cp36-none-any.whl size=50883 sha256=8d4676dfd8b3ba515d9dfc10bc37967fb6552d143fce7693d872a81c62d6499c
  Stored in directory: /root/.cache/pip/wheels/cd/b2/2c/eac6af3343b21f907123ce013d20ad5ad70c2c3731072d98bf
Successfully built pyTelegramBotAPI


In [ ]:
import telebot
from telebot import types
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import copy
import torch.optim.lr_scheduler as sched
from google.colab import files

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)


class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            return (img - self.mean) / self.std


imsize = 478
loader = transforms.Compose([
    transforms.Resize(imsize),  # нормируем размер изображения
    transforms.CenterCrop(imsize),
    transforms.ToTensor()])
unloader = transforms.ToPILImage() # тензор в кратинку
plt.ion()


def gram_matrix(input_img):
    batch_size, f_map_num, h, w = input_img.size()
    features = input_img.view(batch_size * f_map_num, h * w)
    g = torch.mm(features, features.t())
    return g.div(batch_size * h * w * f_map_num)


class ContentLoss(nn.Module):
    def __init__(self, target, ):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
        self.loss = F.mse_loss(self.target, self.target)

    def forward(self, input_img):
        self.loss = F.mse_loss(input_img, self.target)
        return input_img


class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
        self.loss = F.mse_loss(self.target, self.target)

    def forward(self, input_img):
        g = gram_matrix(input_img)
        self.loss = F.mse_loss(g, self.target)
        return input_img


cnn = models.vgg19(pretrained=True).features.to(device).eval()
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']


def get_input_optimizer(input_img):
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer


def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(normalization_mean, normalization_std).to(device)
    content_losses = []
    style_losses = []
    model = nn.Sequential(normalization)

    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses


def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, chat_id, num_steps=400,
                       style_weight=100000, content_weight=1):
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
                                                                     normalization_mean, normalization_std,
                                                                     style_img, content_img)

    optimizer = get_input_optimizer(input_img)
    sheduler = sched.StepLR(optimizer=optimizer, step_size=100, gamma=0.1)

    run = [0]
    while run[0] <= num_steps:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()

            model(input_img)

            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss
            # взвешивание ощибки
            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score

            loss.backward()

            run[0] += 1

            if run[0] % 100 == 0:
                bot.send_message(chat_id, str(int(run[0]/400 * 100)) + ' %')

            return style_score + content_score

        optimizer.step(closure)

        sheduler.step()

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [ ]:
device

device(type='cuda')

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2271, done.
remote: Total 2271 (delta 0), reused 0 (delta 0), pack-reused 2271
Receiving objects: 100% (2271/2271), 8.06 MiB | 5.61 MiB/s, done.
Resolving deltas: 100% (1462/1462), done.


In [ ]:
!bash ./scripts/download_cyclegan_model.sh style_cezanne
!bash ./scripts/download_cyclegan_model.sh style_monet
!bash ./scripts/download_cyclegan_model.sh style_vangogh

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [style_cezanne]
for details.

--2020-07-02 19:01:01--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/style_cezanne.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/style_cezanne_pretrained/latest_net_G.pth’

./checkpoints/style 100%[===================>]  43.46M  14.9MB/s    in 2.9s    

2020-07-02 19:01:04 (14.9 MB/s) - ‘./checkpoints/style_cezanne_pretrained/latest_net_G.pth’ saved [45575747/45575747]



In [ ]:
!pip install dominate

In [ ]:
!ls

checkpoints	docs		 models		requirements.txt  util
CycleGAN.ipynb	environment.yml  options	scripts
data		imgs		 pix2pix.ipynb	test.py
datasets	LICENSE		 README.md	train.py


In [ ]:
!mkdir content
!mkdir style
!mkdir result
!mkdir cezanne
!mkdir monet
!mkdir vangogh

In [ ]:
id_to_stage = {}

token = '1266472120:AAFOzabwzE_i7pesFN9JykneIom5ywpPY-8'
bot = telebot.TeleBot(token) # Токен API к Telegram


@bot.message_handler(commands=['start'])
def start_message(message):
    id_to_stage[message.chat.id] = 'start'
    bot.send_message(message.chat.id, 'Привет, я буду стилизовать фотографии. \
Если ты здесь впервые или тебе нужна помощь, нажми на /help.')
    markup = types.InlineKeyboardMarkup()
    button_cezanne = types.InlineKeyboardButton(text='Cezanne style', 
                                                callback_data='cezanne')
    button_monet = types.InlineKeyboardButton(text='Monet style', 
                                              callback_data='monet')
    button_vangogh = types.InlineKeyboardButton(text='van Gogh style', 
                                                callback_data='vangogh')
    button_custom = types.InlineKeyboardButton(text='Custom style', 
                                               callback_data='custom')
    markup.add(button_cezanne, button_monet, button_vangogh, button_custom)
    bot.send_message(message.chat.id, text='Выбери режим: \nCezanne - стилизация \
под картины Поля Сезанна\nMonet - стилизация под картины Клода Моне\nvan Gogh -\
стилизация под картины Винсента ван Гога\nCustom - \
стилизация под выбранную тобой фотографию',
    reply_markup=markup)


@bot.message_handler(commands=['help'])
def start_message(message):
    chat_id = message.chat.id
    if (chat_id not in id_to_stage):
        id_to_stage[chat_id] = 'start'
    if id_to_stage[chat_id] == 'start':  
        bot.send_message(message.chat.id, text='Выбери режим стилизации /mode')
    elif id_to_stage[chat_id] in ['content', 'cezanne', 'monet', 'vangogh']:
        bot.send_message(chat_id, 'Отправь фотографию, которую я буду стилизовать!')
    elif id_to_stage[chat_id] == 'style':
        bot.send_message(chat_id, 'Отправь фотографию, с которой я буду брать стиль')


@bot.message_handler(commands=['mode'])
def start_message(message):
    chat_id = message.chat.id
    id_to_stage[chat_id] = 'start'
    markup = types.InlineKeyboardMarkup()
    button_cezanne = types.InlineKeyboardButton(text='Cezanne style', 
                                                callback_data='cezanne')
    button_monet = types.InlineKeyboardButton(text='Monet style', 
                                              callback_data='monet')
    button_vangogh = types.InlineKeyboardButton(text='van Gogh style', 
                                                callback_data='vangogh')
    button_custom = types.InlineKeyboardButton(text='Custom style', 
                                               callback_data='custom')
    markup.add(button_cezanne, button_monet, button_vangogh, button_custom)
    
    bot.send_message(chat_id, text='Выбери стиль', 
    reply_markup=markup)


@bot.message_handler(commands=['end'])
def start_message(message):
    chat_id = message.chat.id
    id_to_stage[chat_id] = 'end'
    bot.send_message(chat_id, 'Работа завершена')


@bot.callback_query_handler(func=lambda call: True)
def query_handler(call):
    if call.data == 'cezanne':
        bot.answer_callback_query(callback_query_id=call.id, text='Ты выбрал \
стиль картин Поля Сезанна')
        id_to_stage[call.from_user.id] = 'cezanne'
        bot.send_message(call.from_user.id, 'Выбран стиль Поля Сезанна. \
Отправь фотографию, которую я буду стилизовать!')
    elif call.data == 'monet':
        bot.answer_callback_query(callback_query_id=call.id, text='Ты выбрал \
стиль картин Клода Моне')
        id_to_stage[call.from_user.id] = 'monet'
        bot.send_message(call.from_user.id, 'Выбран стиль Клода Моне. \
Отправь фотографию, которую я буду стилизовать!')
    elif call.data == 'vangogh':
        bot.answer_callback_query(callback_query_id=call.id, text='Ты выбрал \
стиль картин Винсента ван Гога')
        id_to_stage[call.from_user.id] = 'vangogh'
        bot.send_message(call.from_user.id, 'Выбран стиль Винсента ван Гога. \
Отправь фотографию, которую я буду стилизовать!')
    elif call.data == 'custom':
        bot.answer_callback_query(callback_query_id=call.id, text='Ты выбрал \
собственный стиль')
        id_to_stage[call.from_user.id] = 'content'
        bot.send_message(call.from_user.id, 'Отправь фотографию, которую я \
буду стилизовать!')


@bot.message_handler(content_types=["text", "sticker", "pinned_message", "audio"])
def echo_msg(message):
    chat_id = message.chat.id
    if (chat_id not in id_to_stage):
        id_to_stage[message.chat.id] = 'start'
    if message.content_type == 'text':
        if id_to_stage[chat_id] in ['content', 'cezanne', 'monet', 'vangogh']:
            bot.send_message(chat_id, 'Отправь фотографию, которую я буду стилизовать!')
        elif id_to_stage[chat_id] == 'style':
            bot.send_message(chat_id, 'Отправь фотографию, с которой я буду брать стиль!')
        else:
            bot.send_message(chat_id, 'Для начала выбери режим стилизации - /mode')


@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    chat_id = message.chat.id
    if chat_id not in id_to_stage:
        id_to_stage[chat_id] = 'start'
    if id_to_stage[chat_id] == 'start':
        bot.send_message(chat_id, 'Перед тем, как отправлять фотографию \
нужно выбрать режим стилижации - /mode')
    try:
        file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        if id_to_stage[chat_id] in ['content', 'cezanne', 'monet', 'vangogh', 'style']:
            src = id_to_stage[chat_id] + '/' + str(chat_id) + '.jpg'
        else:
            src = file_info.file_path
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
        if id_to_stage[chat_id] == 'content':
            id_to_stage[chat_id] = 'style'
            bot.reply_to(message, "Фото контента добавлено. Теперь отправь фотографию, с которой я буду"
                                  " переносить стиль.")
        elif id_to_stage[chat_id] == 'cezanne':
            id_to_stage[chat_id] = 'start'
            bot.reply_to(message, "Фото добавлено. Начинаю обработку...")
            !python test.py --dataroot cezanne --name style_cezanne_pretrained --model test --no_dropout
            bot.send_photo(chat_id, open('./results/style_cezanne_pretrained/test_latest/images/' + str(chat_id) + '_fake.png', 'rb'))
        elif id_to_stage[chat_id] == 'monet':
            id_to_stage[chat_id] = 'start'
            bot.reply_to(message, "Фото добавлено. Начинаю обработку...")
            !python test.py --dataroot monet --name style_monet_pretrained --model test --no_dropout
            bot.send_photo(chat_id, open('./results/style_monet_pretrained/test_latest/images/' + str(chat_id) + '_fake.png', 'rb'))
        elif id_to_stage[chat_id] == 'vangogh':
            id_to_stage[chat_id] = 'start'
            bot.reply_to(message, "Фото добавлено. Начинаю обработку...")
            !python test.py --dataroot vangogh --name style_vangogh_pretrained --model test --no_dropout
            bot.send_photo(chat_id, open('results/style_vangogh_pretrained/test_latest/images/' + str(chat_id) + '_fake.png', 'rb'))
        elif id_to_stage[chat_id] == 'style':
            id_to_stage[chat_id] = 'start'
            bot.reply_to(message, "Фото стиля добавлено. Начинаю обработку...")
            content_img = image_loader('content/' + str(chat_id) + '.jpg')
            style_img = image_loader('style/' + str(chat_id) + '.jpg')
            input_img = torch.randn(content_img.data.size(), device=device)
            output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std, content_img,
                                                 style_img, input_img, chat_id)
            
            image = output.cpu().clone()   
            image = image.squeeze(0)     
            image = unloader(image)
            image.save('result/' + str(chat_id) + '.jpg')
            bot.send_photo(chat_id, open('result/' + str(chat_id) + '.jpg', 'rb'))

    except Exception as e:
        bot.reply_to(message, e)


bot.polling(none_stop=True, interval=0)

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: cezanne                       	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            